In [9]:
%load_ext autoreload
%autoreload 2
import pynwb
import os
import numpy as np

# DataJoint and DataJoint schema
import datajoint as dj

dj.config["filepath_checksum_size_limit"] = 1 * 1024**2

## We also import a bunch of tables so that we can call them easily
from spyglass.common import (
    ElectrodeGroup,
    Electrode,
    FirFilterParameters,
    IntervalList,
    Session,
    Task,
    TaskEpoch,
    Nwbfile,
    AnalysisNwbfile,
    NwbfileKachery,
    AnalysisNwbfileKachery,
    interval_list_contains,
    interval_list_contains_ind,
    interval_list_excludes,
    interval_list_excludes_ind,
    interval_list_intersect,
    get_electrode_indices,
)

from spyglass.lfp.v1 import (
    LFPElectrodeGroup,
    LFPSelection,
    LFPV1,
)
from spyglass.lfp.lfp_merge import LFPOutput


from spyglass.lfp.analysis.v1.lfp_band import LFPBandSelection,LFPBandV1,FirFilterParameters
# from spyglass.lfp_band.lfp_band_merge import LFPBandOutput
import os
import warnings
from tqdm import tqdm
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=ResourceWarning)
warnings.simplefilter("ignore", category=UserWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import os
os.chdir('/home/sambray/Documents/MS_analysis_samsplaying/')
from ms_opto_stim_protocol import OptoStimProtocol
set((OptoStimProtocol() & {'optogenetic_protocol':'pulse_train'}).fetch('period_ms'))
FirFilterParameters() & {'filter_name':filter_name}

filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,filter_type,filter_low_stop lowest frequency for stop band for low frequency side of filter,filter_low_pass lowest frequency for pass band of low frequency side of filter,filter_high_pass highest frequency for pass band for high frequency side of filter,filter_high_stop highest frequency for stop band of high frequency side of filter,filter_comments comments about the filter,filter_band_edges numpy array containing the filter bands (redundant with individual parameters),filter_coeff numpy array containing the filter coefficients
ms_stim_125ms_period,1000,bandpass,6.5,7.0,9.0,9.5,,=BLOB=,=BLOB=


In [31]:
#make a filter for each driving period
period_ms = 125
for period_ms in [250,165,125,100,80,50]:
    filter_name = f"ms_stim_{period_ms}ms_period"
    filter_type = 'bandpass'
    sampling_rate = 1000
    center_freq = 1/(period_ms/1000)
    bandwidth = 1
    band_edges = [center_freq - bandwidth * 1.5 ,
                center_freq - bandwidth,
                center_freq + bandwidth,
                center_freq + bandwidth * 1.5]
    comment = f"ms stim project. {period_ms}ms period driving frequency"

    FirFilterParameters().add_filter(filter_name=filter_name,
                                filter_type=filter_type,
                                fs=sampling_rate,
                                band_edges=band_edges,)

## Define Animal Sessions

In [27]:
datset = (OptoStimProtocol() & {'optogenetic_protocol':'pulse_train'})

datset.fetch('nwb_file_name','interval_list_name','period_ms')

[array(['Banner20211221_.nwb', 'Banner20211221_.nwb',
        'Banner20211222_.nwb', 'Banner20211222_.nwb',
        'Banner20211222_.nwb', 'Banner20211222_.nwb',
        'Banner20211222_.nwb', 'Banner20211227_.nwb',
        'Banner20211227_.nwb', 'Banner20211227_.nwb',
        'Banner20211227_.nwb', 'Banner20211227_.nwb',
        'Banner20211228_.nwb', 'Banner20211228_.nwb',
        'Banner20211228_.nwb', 'Banner20211228_.nwb',
        'Banner20211228_.nwb', 'Banner20211228_.nwb',
        'Banner20211228_.nwb', 'Banner20211228_.nwb',
        'Banner20211228_.nwb', 'Banner20211228_.nwb',
        'Banner20211230_.nwb', 'Banner20211230_.nwb',
        'Banner20211230_.nwb', 'Banner20211230_.nwb',
        'Banner20211230_.nwb', 'Banner20211231_.nwb',
        'Banner20211231_.nwb', 'Banner20211231_.nwb',
        'Banner20211231_.nwb', 'Banner20211231_.nwb',
        'Banner20220103_.nwb', 'Banner20220103_.nwb',
        'Banner20220103_.nwb', 'Banner20220103_.nwb',
        'Banner20220103_.nwb

## Populate

In [47]:
# desired sampling rate
lfp_band_sampling_rate = 200
missing_lfp = []


for nwb_file_name, interval_list_name, period_ms in zip(*datset.fetch('nwb_file_name','interval_list_name','period_ms')):
    if period_ms not in [250,165,125,100,80,50]:
        continue
    
    filter_name = f"ms_stim_{period_ms}ms_period" 
            
    basic_key = {"nwb_file_name": nwb_file_name,'target_interval_list_name':interval_list_name}
    
    #check that lfp exists for this animal
    if len(LFPV1() & basic_key)==0:
        missing_lfp.append(basic_key)
        continue
    if len(LFPV1 & basic_key)>1:
        electrode_group_names = (LFPV1() & basic_key).fetch("lfp_electrode_group_name")
        electrode_group_name =[x for x in electrode_group_names if nwb_file_name.split('202')[0] in x][0]
        basic_key["lfp_electrode_group_name"] = electrode_group_name
    if not (LFPV1 & basic_key).fetch1('lfp_sampling_rate')==1000:
        continue
        
    #get lfp's sampling rate
    lfp_sampling_rate = (LFPV1() & basic_key ).fetch1(
        "lfp_sampling_rate"
    )
    
    #get lfp filter sampling_rate
    lfp_filter_sampling_rate = (LFPV1() & basic_key).fetch1(
        "filter_sampling_rate"
    )
    
    #get lfp filter name
    lfp_filter_name = (LFPV1() & basic_key).fetch1(
        "filter_name"
    )
    
    # make lfp selection key
    lfp_s_key = basic_key.copy()
    lfp_s_key["target_interval_list_name"] = interval_list_name
    lfp_s_key["filter_name"] = lfp_filter_name
    lfp_s_key["filter_sampling_rate"] = lfp_filter_sampling_rate
    
    # get the electrodes the lfp was run on
    lfp_electrode_group_name = (LFPV1 & lfp_s_key).fetch1('lfp_electrode_group_name')
    lfp_eg_key = {
            "nwb_file_name": nwb_file_name,
            "lfp_electrode_group_name": lfp_electrode_group_name,
            }
    electrode_list = list(np.asarray((LFPV1 & basic_key).fetch_nwb()[0]['lfp'].electrodes.to_dataframe().index))
    
    # remake lfp selection key
    lfp_s_key = basic_key.copy()
    lfp_s_key["target_interval_list_name"] = interval_list_name
    lfp_s_key["filter_name"] = lfp_filter_name
    lfp_s_key["filter_sampling_rate"] = lfp_filter_sampling_rate
    
    # set the reference to -1 to indicate no reference for all channels
    ref_elect = [-1]
    
    # we also need the uuid for the LFP object
    lfp_merge_id = (LFPOutput.LFPV1 & lfp_s_key).fetch1('merge_id')
    
    
    # #make Band selection table entry
    LFPBandSelection().set_lfp_band_electrodes(
            nwb_file_name=nwb_file_name,
            lfp_merge_id=lfp_merge_id,
            electrode_list=electrode_list,
            filter_name=filter_name,
            interval_list_name=interval_list_name,
            reference_electrode_list=ref_elect,
            lfp_band_sampling_rate=lfp_band_sampling_rate,
        )
    
    lfp_band_key = (
        LFPBandSelection
        & {
            "lfp_merge_id": lfp_merge_id,
            "filter_name": filter_name,
            "lfp_band_sampling_rate": lfp_band_sampling_rate,
            "target_interval_list_name":interval_list_name,
        }
    ).fetch1("KEY")
    if LFPBandV1() & lfp_band_key:
        continue #already populated
    # lfp_band_key['lfp_electrode_group_name']="full_probe"
    LFPBandV1().populate(LFPBandSelection()&lfp_band_key)

[2023-11-08 09:18:38,801][WARNING]: Skipped checksum for file with hash: 6cd0fdb9-514d-d485-4dd4-bd1421c2a72f, and path: /stelmo/nwb/analysis/Banner20211221/Banner20211221_B8Y8EIB8TP.nwb
[2023-11-08 09:18:39,185][WARNING]: Skipped checksum for file with hash: 06436bdb-6d9c-22f6-029e-88c2f8c2412d, and path: /stelmo/nwb/analysis/Banner20211221/Banner20211221_FU6J57GS4G.nwb
[2023-11-08 09:18:39,558][WARNING]: Skipped checksum for file with hash: ff3b5c39-9487-8d16-a69c-43084265943b, and path: /stelmo/nwb/analysis/Banner20211222/Banner20211222_24ZSISUYXR.nwb
[2023-11-08 09:18:39,944][WARNING]: Skipped checksum for file with hash: 52940a6b-3512-2d5a-bd6d-715de04c937e, and path: /stelmo/nwb/analysis/Banner20211222/Banner20211222_NJ1AH2SG7X.nwb
[2023-11-08 09:18:40,335][WARNING]: Skipped checksum for file with hash: 18e8343e-3700-869b-51a4-ac5a8fae20c4, and path: /stelmo/nwb/analysis/Banner20211222/Banner20211222_PATV8XFL7F.nwb
[2023-11-08 09:18:40,743][WARNING]: Skipped checksum for file wit

Writing new NWB file Olive20220628_HOHNUWIJFV.nwb


[2023-11-08 09:19:31,808][WARNING]: Skipped checksum for file with hash: 82309055-c69e-e1d2-0f42-3f528c8c24a0, and path: /stelmo/nwb/analysis/Olive20220628/Olive20220628_DERT5IYUCB.nwb
[2023-11-08 09:19:32,582][WARNING]: Skipped checksum for file with hash: 82309055-c69e-e1d2-0f42-3f528c8c24a0, and path: /stelmo/nwb/analysis/Olive20220628/Olive20220628_DERT5IYUCB.nwb


Writing new NWB file Olive20220628_20DEW7ECIX.nwb


[2023-11-08 09:19:39,745][WARNING]: Skipped checksum for file with hash: 75cec1be-66e1-eb6c-f045-293a2a5177e2, and path: /stelmo/nwb/analysis/Olive20220628/Olive20220628_ZK9FKM7URM.nwb
[2023-11-08 09:19:40,565][WARNING]: Skipped checksum for file with hash: 75cec1be-66e1-eb6c-f045-293a2a5177e2, and path: /stelmo/nwb/analysis/Olive20220628/Olive20220628_ZK9FKM7URM.nwb


...

Writing new NWB file Yoshi20220511_GSIQYM4ISE.nwb


[2023-11-08 09:46:24,071][WARNING]: Skipped checksum for file with hash: 1efc519d-3e95-ff70-5cb4-ee97280ff168, and path: /stelmo/nwb/analysis/Yoshi20220511/Yoshi20220511_5JAVDO89OW.nwb
[2023-11-08 09:46:25,317][WARNING]: Skipped checksum for file with hash: 1efc519d-3e95-ff70-5cb4-ee97280ff168, and path: /stelmo/nwb/analysis/Yoshi20220511/Yoshi20220511_5JAVDO89OW.nwb


Writing new NWB file Yoshi20220511_BNKO63PFVW.nwb


[2023-11-08 09:46:33,705][WARNING]: Skipped checksum for file with hash: eb9cea12-1442-bda9-7c73-1c281ebe4635, and path: /stelmo/nwb/analysis/Yoshi20220511/Yoshi20220511_G8J9DWJE1I.nwb
[2023-11-08 09:46:34,741][WARNING]: Skipped checksum for file with hash: eb9cea12-1442-bda9-7c73-1c281ebe4635, and path: /stelmo/nwb/analysis/Yoshi20220511/Yoshi20220511_G8J9DWJE1I.nwb


Writing new NWB file Yoshi20220511_XZASZ9MA49.nwb


In [46]:
basic_key
period_ms
(LFPV1 & basic_key)#.fetch1('lfp_sampling_rate')
LFPV1() & {'nwb_file_name':'Winnie20220713_.nwb'}
animal = Session & {'nwb_file_name':nwb_file_name}.fetch1('subject_id')
electrode_group_name = f'tetrode_sample_{animal}'


nwb_file_name name of the NWB file,lfp_electrode_group_name the name of this group of electrodes,target_interval_list_name descriptive name of this interval list,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,analysis_file_name name of the file,interval_list_name descriptive name of this interval list,lfp_object_id the NWB object ID for loading this object from the file,"lfp_sampling_rate the sampling rate, in HZ"
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 0 valid times,LFP 0-400 Hz,30000,Winnie20220713_C52XDICU6D.nwb,lfp_tetrode_sample_Winnie_pos 0 valid times_valid times,a89c590f-290b-4f9c-a568-b9ae67eee96d,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 1 valid times,LFP 0-400 Hz,30000,Winnie20220713_FURB0FGK1A.nwb,lfp_tetrode_sample_Winnie_pos 1 valid times_valid times,2d99415e-6646-44e1-8745-d0af2912f9b1,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 10 valid times,LFP 0-400 Hz,30000,Winnie20220713_9GNIXRYWTS.nwb,lfp_tetrode_sample_Winnie_pos 10 valid times_valid times,48bc053f-ff5b-460f-838b-cefd8eba4b0d,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 11 valid times,LFP 0-400 Hz,30000,Winnie20220713_824JXQAB78.nwb,lfp_tetrode_sample_Winnie_pos 11 valid times_valid times,87e1b640-4938-4974-bce2-766f61c2e66f,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 12 valid times,LFP 0-400 Hz,30000,Winnie20220713_FLOILA8D0V.nwb,lfp_tetrode_sample_Winnie_pos 12 valid times_valid times,21aa2368-261b-4a1e-9e3b-151727e7a8b9,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 13 valid times,LFP 0-400 Hz,30000,Winnie20220713_YYUM4C0GTJ.nwb,lfp_tetrode_sample_Winnie_pos 13 valid times_valid times,599008ed-d7b4-4aca-9a65-b7cb6619a026,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 14 valid times,LFP 0-400 Hz,30000,Winnie20220713_49PBQ0QLK5.nwb,lfp_tetrode_sample_Winnie_pos 14 valid times_valid times,5aac489d-2f02-49eb-92e3-46d31390c193,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 2 valid times,LFP 0-400 Hz,30000,Winnie20220713_HBKDTJIJAR.nwb,lfp_tetrode_sample_Winnie_pos 2 valid times_valid times,3e8e3eaf-a12d-4b0f-b218-6e2deac10f30,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 3 valid times,LFP 0-400 Hz,30000,Winnie20220713_I2WK0NS3K8.nwb,lfp_tetrode_sample_Winnie_pos 3 valid times_valid times,e7e39ca8-98e9-4dc2-bded-2a0b2980ff07,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 4 valid times,LFP 0-400 Hz,30000,Winnie20220713_W4TKS379ID.nwb,lfp_tetrode_sample_Winnie_pos 4 valid times_valid times,bf864bf0-6b1a-4043-a34a-65f036dd3e27,1000.0


In [10]:
lfp_band_key

(LFPV1 & basic_key).fetch_nwb()[0]['lfp']

[2023-10-20 08:18:10,755][WARNING]: Skipped checksum for file with hash: 1f41ed1f-df73-533d-6c95-5cb459607516, and path: /stelmo/nwb/analysis/Bilbo20230725/Bilbo20230725_JRH9I6411O.nwb


filtered data pynwb.ecephys.ElectricalSeries at 0x140401433184672
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (2016507, 14), type "<i2">
  description: filtered data
  electrodes: electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  interval: 1
  offset: 0.0
  resolution: -1.0
  timestamps: <HDF5 dataset "timestamps": shape (2016507,), type "<f8">
  timestamps_unit: seconds
  unit: volts

## Check missing LFP's

In [13]:
print(missing_lfp)

'Banner20220225_.nwb'